In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pytest

import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd

import FINE as fn

In [50]:
component_list = ['c1','c2']  
space_list = ['01_reg','02_reg','03_reg','04_reg','05_reg']

in_data = np.array([ [np.nan for i in range(5)],
                       [1, 1, 1, 0, 0] 
                   ])

xr_in = xr.DataArray(in_data, 
                   coords=[component_list, space_list], 
                   dims=['component','space'])    

In [51]:
reg_groups_dict = {'01_reg_02_reg_03_reg' : ['01_reg','02_reg','03_reg'], '04_reg_05_reg': ['04_reg','05_reg']}

In [52]:
component_list = ['c1','c2']  
space_list = list(reg_groups_dict.keys())

out_data = np.zeros((len(component_list), len(space_list)))

xr_out = xr.DataArray(out_data, 
                       coords=[component_list, space_list], 
                       dims=['component','space'])    

In [53]:
xr_weight = xr_in

In [54]:
for grouped_reg, reg_list in reg_groups_dict.items():
    weighted_xr = xr_in * xr_weight

    xr_out.loc[dict(space=grouped_reg)
    ] = weighted_xr.sel(space=reg_list).sum(
            dim='space', skipna=False
    ) / xr_weight.sel(
        space=reg_list
    ).sum(
        dim='space', skipna=False
    )
        

In [63]:
for comp in xr_out.component:
    if not xr_out.loc[comp].isnull().all() and xr_out.loc[comp].isnull().any():
        xr_out.loc[comp] = (xr_out.loc[comp]).fillna(0)

In [64]:
xr_out

<xarray.DataArray (component: 2, space: 2)>
array([[nan, nan],
       [ 1.,  0.]])
Coordinates:
  * component  (component) <U2 'c1' 'c2'
  * space      (space) <U20 '01_reg_02_reg_03_reg' '04_reg_05_reg'

In [ ]:
xr

In [ ]:
component_list = ['c1','c2']  
space_list = ['01_reg','02_reg','03_reg','04_reg','05_reg']

data = np.array([ [np.nan for i in range(5)],
                       [3.0, 3.0, 3.0, 0.0, 0.0] 
                   ])

xr = xr.DataArray(data, 
                   coords=[component_list, space_list], 
                   dims=['component','space'])  

In [ ]:
xr.where(np.logical_or(xr.isnull(), xr==0), 1)

In [49]:
0/0

ZeroDivisionError: division by zero